In [ ]:
!pip install -q openai

In [ ]:
import os
try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
except ImportError:
    pass  # Not running in Colab; uses env var already set


# Exercise 6: The Agentic Loop

In this exercise, we put it all together into an **agent loop** — the core pattern behind AI agents.

By the end, you'll understand:
- How to build a loop that keeps running until the model stops requesting tools
- How to dispatch tool calls to actual functions
- How the model can chain **multiple** tool calls to answer a single question

## Step 1: Setup

In [ ]:
import json
import openai
from datetime import datetime

client = openai.OpenAI()

## Step 2: Define multiple tools and a dispatcher

This time we have **two** tools. The `call_tool` function maps tool names to their implementations — this is how real agents work.

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    }
                },
                "required": ["location"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_time",
            "description": "Get the current date and time",
            "parameters": {
                "type": "object",
                "properties": {},
            },
        },
    },
]


def call_tool(tool_name, tool_input):
    if tool_name == "get_weather":
        return f"72°F and sunny in {tool_input['location']}"
    elif tool_name == "get_time":
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    else:
        return "Tool not found."

## Step 3: The agent loop

This is the core pattern. The loop:
1. Sends messages to the model
2. Checks if the response contains any `tool_calls`
3. If yes — executes each tool, appends results, and loops back to step 1
4. If no — the model is done, we print the final text and break

Because we ask about **both** weather and time, the model needs to call two tools — watch the loop iterate!

In [ ]:
messages = [{"role": "user", "content": "What's the weather and current time in Tokyo?"}]

loop_count = 0

while True:
    loop_count += 1
    print(f"--- Loop iteration {loop_count} ---")

    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=messages,
        tools=tools,
    )

    message = response.choices[0].message
    print(f"finish_reason: {response.choices[0].finish_reason}")

    # Add assistant response to messages
    messages.append(message)

    if message.tool_calls:
        for tool_call in message.tool_calls:
            args = json.loads(tool_call.function.arguments)
            tool_result = call_tool(tool_call.function.name, args)
            print(f"  Tool: {tool_call.function.name}({args}) → {tool_result}")
            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": tool_result,
                }
            )
    else:
        print(f"\n{message.content}")
        print("\nDone! No more tool calls.")
        break

## Step 4: Review the full message history

In [ ]:
for msg in messages:
    if hasattr(msg, 'role'):
        role = msg.role
        content = msg.content or f"[tool_calls: {[tc.function.name for tc in msg.tool_calls]}]"
    else:
        role = msg['role']
        content = msg.get('content', '')
    print(f"--- {role} ---")
    print(f"{content}\n")

## Key takeaway

This is the **agentic loop** — the foundation of every AI agent:

```
while True:
    response = call_model(messages)
    if response has tool calls:
        execute tools, append results
    else:
        break  # model is done
```

Everything else — multiple tools, error handling, human-in-the-loop, memory — is built on top of this pattern.

Now let's build some real agents!